## ANN with Regression for Salary prediction
### Using salary as continuous faeture for prediction

In [1]:
## Required modules

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [2]:
## Get data set

data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
## EstimatedSalary will be the output(dependent) feature while all other will independent features
## As this output feature has continuous values to use for regression
## Delete unnecessary features

data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [ ]:
## For Gender Label Encoder for only two categories
label_encoder_gender = LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

## It will conver male and female into 1 and 0
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [5]:
## But in Geography	column, there are more than two categories
## Here will use OneHotEncoder instead of LabelEncoder

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [6]:
## Now combine all onehot encoded columns in the original data
## First drop 'Geography' column and then concate with geo_encoded_df

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [7]:
## Split the data into features ans target
## X without EstimatedSalary and all other columns
## y wiyh only EstimatedSalary column
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

In [8]:
## Split the data in training and testing sets (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
## Scale thsese features from 0 to 1
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
## Save the encoder and scalers
## Save this file in the pickle containing with both LabelEncoder and OnhotEncoder
## This pickle file will be used in future
## Save this file in the form of write byte mode (wb)
## Use dump to convert into json

with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

### Train ANN with Regression Problem Statementm

In [11]:
import tensorflow as tf

## To use sequential model
from tensorflow.keras.models import Sequential
## To create nodes
from tensorflow.keras.layers import Dense
## For early stopping and for visuals
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
## To use date time
import datetime

In [12]:
## Build the ANN Model
## Create and initialize sequential model
## Here we will use 64 nodes/neurons in hidden layer and apply relu activation function
## Only in first hidden layer (HL1), needs to write input_shape to make connections between input and Dense layer
model = Sequential([
    ## HL1 connected with input layer
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), 
    ## HL2, Don't need input_shape, now it knows that it will be connected already
    Dense(32, activation='relu'),
    ## This is Output layer with Dense 1, here not applying any activation function
    ## using default activation function called linear which is specific for regression      
    Dense(1) 
])

## Compile the model for regression

## Can use either mean absolute error (mae) or mean square error (mse) for loss. 
## For more info on loss functions, https://keras.io/api/losses/
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

model.summary()

/Users/mak25/Documents/udem_genai_mk/ml_dl_projects/mldl-env/lib/python3.12/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
## To train this model
## Set up TensorBoard

log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [14]:
## Early stoping

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [16]:
## Finally start model training by using callbacks
history = model.fit(
    X_train, y_train, 
    validation_data=(X_test, y_test), 
    epochs=100,
    callbacks=[tensorboard_callback, early_stopping_callback]
    )

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 100393.4922 - mae: 100393.4922 - val_loss: 98575.6797 - val_mae: 98575.6797
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 658us/step - loss: 99840.4688 - mae: 99840.4688 - val_loss: 97446.4531 - val_mae: 97446.4531
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 661us/step - loss: 97850.1250 - mae: 97850.1250 - val_loss: 94501.6094 - val_mae: 94501.6094
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 649us/step - loss: 93834.0938 - mae: 93834.0938 - val_loss: 89432.9688 - val_mae: 89432.9688
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 645us/step - loss: 87766.8047 - mae: 87766.8047 - val_loss: 82622.0000 - val_mae: 82622.0000
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 653us/step - loss: 80196.7344 - mae: 80196.7344 - val_loss: 74855.3906 - val_mae: 74855.3906
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 651us/step - loss: 72218.4141 - mae: 72218.4141 - val_loss: 67388.8438 - val_mae: 67388.8438
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 

In [17]:
### Load Tensorboard Extension

%load_ext tensorboard

In [19]:
## Launch Tensorboard to see logs visuals by prividing newly created fit file name in logs

%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 1812), started 0:00:13 ago. (Use '!kill 1812' to kill it.)

In [ ]:
## Use '!kill 1812' to stop TensorBoard, using port id (PID) number with !kll command

!kill 1812

In [21]:
## Evaluate model on the test data to check performance

test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE: {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 524us/step - loss: 50315.3672 - mae: 50315.3672
Test MAE: 50315.3671875


In [22]:
## Save this model in h5 file

model.save('regression_model.h5')

In [ ]:
## This can be deployed to Strealit cloud by creating ann_regression_app.py file